In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import time
import seaborn as sns
import shutil
from torch import optim
from torch.nn import functional as F
from IPython.display import clear_output
from IPython.utils import io

with io.capture_output() as captured:
  !pip install transformers sentencepiece sentence_transformers

from sentence_transformers import SentenceTransformer, SentencesDataset, LoggingHandler, losses
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import SentenceEvaluator
from torch.utils.data import DataLoader
from datetime import datetime
import math

from transformers import AdamW, AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm

with io.capture_output() as captured:
  !pip install lingtrain_aligner dateparser razdel

from lingtrain_aligner import preprocessor, splitter, aligner, resolver, reader, helper, vis_helper, metrics

sns.set()

Load texts in both languages. To do all texts, the notebook have to be runned with every text path

In [ ]:
with open('birth_rus.txt', 'r') as file:
    text1 = file.read()

In [ ]:
with open('birth_roma.txt', 'r') as file:
    text2 = file.read()

In [ ]:
lang_from = "roma"
lang_to = "ru"
db_path = "alignment.db"

use_proxy_from = False
use_proxy_to = False

In [ ]:
splitted_from = splitter.split_by_sentences(text1.split('\n'), lang_from)
splitted_to = splitter.split_by_sentences(text2.split('\n'), lang_to)

aligner.fill_db(db_path, lang_from, lang_to, splitted_from, splitted_to)

In [ ]:
batch_ids = [0,1,2,3]

shift = 0
window = 80

In [ ]:
aligner.align_db(db_path,
                model_name="sentence_transformer_multilingual_labse",
                batch_size=200,
                window=window,
                batch_ids=batch_ids,
                save_pic=False,
                embed_batch_size=5,
                normalize_embeddings=True,
                show_progress_bar=True,
                shift=shift,
                model=model,
                use_proxy_from=use_proxy_from,
                use_proxy_to=use_proxy_to
                )

In [ ]:
steps = 3
batch_id = -1
for i in range(steps):
    conflicts, rest = resolver.get_all_conflicts(db_path, min_chain_length=2+i, max_conflicts_len=6*(i+1), batch_id=batch_id)
    resolver.get_statistics(conflicts)
    resolver.get_statistics(rest)

    resolver.resolve_all_conflicts(db_path, conflicts, model_name, show_logs=False, model=model)

In [ ]:
conflicts, rest = resolver.get_all_conflicts(db_path, min_chain_length=2, max_conflicts_len=26, batch_id=batch_id)
resolver.get_statistics(conflicts)
resolver.get_statistics(rest)

resolver.resolve_all_conflicts(db_path, conflicts, model_name, show_logs=False, model=model)

In [ ]:
from lingtrain_aligner import saver

batch_ids=[]

output_path="/content"

saver.save_plain_text(db_path, os.path.join(output_path, f"corpora_{lang_from}.txt"), direction="from", batch_ids=batch_ids)
saver.save_plain_text(db_path, os.path.join(output_path, f"corpora_{lang_to}.txt"), direction="to", batch_ids=batch_ids)

saver.save_tmx(db_path, os.path.join(output_path, f"corpora_{lang_from}_{lang_to}.tmx"), lang_from, lang_to)

In [ ]:
from lingtrain_aligner import helper, aligner

lines_from, lines_to = helper.read_processing(db_path)

print(len(lines_from), len(lines_to))

In [ ]:
vec_from = aligner.get_line_vectors(lines_from, model_name="", model=model)
vec_to = aligner.get_line_vectors(lines_to, model_name="", model=model)

In [ ]:
from scipy import spatial

sims = [1 - spatial.distance.cosine(x, y) for x,y in zip(vec_from, vec_to)]

Save aligned pairs to csv

In [ ]:
source = "birth"

In [ ]:
output = {"to": lines_to, "from": lines_from, "sim": sims, "source": source}

In [ ]:
output = pd.DataFrame.from_dict(output)

In [ ]:
output.to_csv(source+".csv")